In [318]:
from math import ceil
import time
import re
import ast
import sys
import urllib
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.error import URLError
import requests
from itertools import cycle
import string
from header_list import user_agent_list
from proxies_list import proxies_list_, proxie_random_pool
from FUNCTIONS import proxie_check, strip_count, gen_url_list
from SCRAPPER_FUNCTIONS import session_creator, zip_prop_count, each_page, links_for_props

In [319]:
ua = user_agent_list
proxies = proxies_list_
prx_pool = proxie_random_pool

In [320]:
# proxie_check(proxies)
len(proxies)

48

In [204]:
zip_codes = pd.read_csv('../Data/main_zips.csv')
zip_list = zip_codes['zip'].tolist()

In [264]:
# active_url_list, sold_url_list = gen_url_list(zip_list)

In [213]:
# a_main_df = pd.DataFrame(columns=['full_address', 'home_link'])
# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

In [261]:
# a = randint(0, 3)
# time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
# active_url_list, a_main_df, proxies = links_for_props(
#     proxies, active_url_list, a_main_df, ua)

In [307]:
# a_main_df

In [310]:
# len(active_url_list)

In [309]:
# a = randint(0, 3)
# sold_url_list, s_main_df, proxies = links_for_props(
#     proxies, sold_url_list, s_main_df, ua)

In [160]:
# sld_under_url_list = sld_under_url_list[1:]
# print(len(sld_under_url_list))

In [614]:
# sld_under_url_list, s_main_df = links_for_props(
#     proxies, sld_under_url_list, s_main_df, ua)

In [616]:
# list_ = list(s_main_df.full_address.values)
# set([num[-5:] for num in list_])

In [612]:
# main_df.to_csv('../Data/main_df_ap.csv')
# s_main_df.to_csv('../Data/s_main.csv')

In [21]:
# s_main_df = pd.read_csv('../Data/s_main.csv')
# main_df = pd.read_csv('../Data/main_df_ap.csv')

In [295]:
a_main_df.to_csv('../Data/active_props_url_list.csv')
s_main_df.to_csv('../Data/sold_props_url_list.csv')

In [304]:
a_main_df = a_main_df.drop_duplicates(subset='full_address')
s_main_df = s_main_df.drop_duplicates(subset='full_address')

In [305]:
active_url_list = a_main_df.home_link.tolist()

In [306]:
len(active_url_list)

10153

In [472]:
def each_property(url, hdr, proxy):
    soup = session_creator(proxy, ua, url)

    price_regex = r'\$\S+\s+'
    bed_regex = r'\d+\S?\d?\d?Bed'
    bath_regex = r'\d+\S?\d?\d?Bath'
    size_regex = r'\d+\S?\d?\d?\s?Sq'
    yr_blt_regex = r'Built: \d+'
    status_regex = r'Status: \w+'
    home_dict = dict()

    home_address_MLS = soup.title.string
    add_list = home_address_MLS.split(sep=' |')
    zip_code = re.search('\d+$', add_list[0])
    mls_num = re.search('\d+$', add_list[1])
    home_add = re.search('(.+?),', home_address_MLS)
    home_city = re.search(',(.+?),', home_address_MLS)
    home_state = add_list[0].replace(home_add.group(1), '').replace(
        home_city.group(1), '').replace(zip_code.group(), '').replace(
        ',', '').replace(' ', '')

    home_dict['address'] = home_add.group(1)
    home_dict['city'] = home_city.group(1)
    home_dict['zip_code'] = zip_code.group()
    home_dict['state'] = home_state
    home_dict['mls_num'] = mls_num.group()

    all_info = soup.findAll('div', {'data-rf-test-id': 'publicRecords'})
    label_list = []
    values_list = []
    for num in all_info:
        cats = num.findAll('span', {'class': 'table-label'})
        for i in cats:
            label_list.append(i.text)
    for num in all_info:
        vals = num.findAll('div', {'class': 'table-value'})
        for i in vals:
            values_list.append(i.text)
    new_dict = dict(zip(label_list, values_list))
    
    
#     all_top = soup.findAll('div', {'class': 'HomeInfo inline-block'})
#     price_list = []
#     for num in all_top:
#         a = num.text
#         price_list.extend(re.findall(price_regex, a))
#         home_dict['num_beds'] = float((re.findall(bed_regex, a))[0].replace(
#             'Bed', '').replace(' ', ''))
#         home_dict['num_baths'] = float((re.findall(bath_regex, a))[0].replace(
#             'Bath', '').replace(' ', ''))
#         home_dict['home_sqft'] = float((re.findall(size_regex, a))[0].replace(
#             'Sq', '').replace(' ', '').replace(':', ''))
#         home_dict['yr_blt'] = int((re.findall(yr_blt_regex, a))[0].replace(
#             'Built', '').replace(' ', '').replace(':', ''))
#         home_dict['status'] = (re.findall(status_regex, a))[0].replace(
#             'Status', '').replace(' ', '').replace(':', '')

    home_dict['redfin_est'] = float([num for num in price_list if 'Redfin' in num
                                     ][0].replace('Redfin ', '').replace('$', '').replace(',', ''))
    home_dict['sold_price'] = float([num for num in price_list if 'Last' in num
                                     ][0].replace('Last ', '').replace('$', '').replace(',', ''))

    home_description = soup.find('p', {'class': 'font-b1'})
    home_dict['description'] = home_description.span.text

    all_home_feats = soup.findAll('span', {'class': "entryItemContent"})
    feats = [num.text for num in all_home_feats]

#     prop_hist = soup.findAll(
#         'div', {'id': 'propertyHistory-expandable-segment'})
#     prop_history = [num.text for num in prop_hist]

    school_info = soup.findAll('div', {'class': "name-and-info"})
    schools = [num.text for num in school_info]

    if ('Public') in schools[0]:
        es = schools[0]
        elementary_school = es.split(sep=' •')
        home_dict['elem_school_name'] = elementary_school[0][:-6]
        home_dict['elem_school_grades'] = elementary_school[1]
        home_dict['elem_school_rating'] = re.findall('(\d+)', elementary_school[2])[0]
    else:
        es = schools[1]
        elementary_school = es.split(sep=' •')
        home_dict['elem_school_name'] = elementary_school[0][:-6]
        home_dict['elem_school_grades'] = elementary_school[1]
        home_dict['elem_school_rating'] = re.findall('(\d+)', elementary_school[2])[0]
    
    if (('Middle' or 'Junior') and 'Public') in schools[1]:
        ms = schools[1]
        middle_school = ms.split(sep=' •')
        home_dict['middle_school_name'] = middle_school[0][:-6]
        home_dict['middle_school_grades'] = middle_school[1]
        home_dict['middle_school_rating'] = re.findall('(\d+)', middle_school[2])[0]
    else:
        ms = schools[2]
        middle_school = ms.split(sep=' •')
        home_dict['middle_school_name'] = middle_school[0][:-6]
        home_dict['middle_school_grades'] = middle_school[1]
        home_dict['middle_school_rating'] = re.findall('(\d+)', middle_school[2])[0]
        
    home_info = {**home_dict, **all_info}

#     if ('9 to 12' and 'Public') in schools[2]:
#         hs = schools[2]
#         high_school = hs.split(sep=' •')
#         home_dict['high_school_name'] = high_school[0][:-6]
#         home_dict['high_school_grades'] = high_school[1]
#         home_dict['high_school_rating'] = high_school[2]
#     else:
#         hs = schools[3]
#         high_school = hs.split(sep=' •')
#         home_dict['high_school_name'] = high_school[0][:-6]
#         home_dict['high_school_grades'] = high_school[1]
#         home_dict['high_school_rating'] = high_school[2]



    return home_info, feats

In [473]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
sold_url_list = s_main_df['home_link'].tolist()
# url = random.sample(active_url_list, 1)[0]
url = random.sample(sold_url_list, 1)[0]
print(proxy)
print(url)

home_dict, feats, all_info = each_property(url, header, proxy)

108.29.37.161:59481
https://www.redfin.com/TX/Frisco/8451-Moore-St-75034/home/31654766


In [499]:
label_list = []
values_list = []
for num in all_info:
    cats = num.findAll('span', {'class': 'table-label'})
    for i in cats:
        label_list.append(i.text)
for num in all_info:
    vals = num.findAll('div', {'class': 'table-value'})
    for i in vals:
        values_list.append(i.text)
new_dict = dict(zip(label_list, values_list))
print({**home_dict, **new_dict})

{'address': '8451 Moore St', 'city': ' Frisco', 'zip_code': '75034', 'state': 'TX', 'num_beds': 4.0, 'num_baths': 4.0, 'home_sqft': 210.0, 'yr_blt': 2007, 'status': 'Sold', 'redfin_est': 380559.0, 'sold_price': 379000.0, 'description': 'Luxury Townhome in Frisco Square! Grand entry with two story ceiling and iron spindle staircase. Open layout with stunning finish out including extensive wood flooring on first floor, tall ceilings, and decorative lighting. Gourmet kitchen offers granite counter tops, stainless appliances, gas cook top and ample storage space. Open family room with built-in entertainment center and stone fireplace. Huge master suite with spa like bath and walk-in closet. Spacious game room and oversized bedrooms all with private bathrooms. 2 Car Garage! Luxury amenities and carefree living in the heart of Frisco. Easy hwy access and shops  &  entertainment within walking distance! ', 'elem_school_name': 'Bright Elementary School', 'elem_school_grades': ' K to 5', 'elem_

In [478]:
all_info

[<div class="sectionContainer" data-rf-test-id="publicRecords"><h2 class="h2 inline-block">Public Facts<span class="adr" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span> for </span><span class="street-address" itemprop="streetAddress">8451 Moore Street</span></span></h2><div class="main-content"><div class="public-records-taxes"><div class="taxable-value"><h3 class="h3">Taxable Value</h3><div class="tax-values"><div class="tax-table"><table><tbody><tr><td class="heading">Land</td><td class="value">N/A</td></tr><tr><td class="heading">Additions</td><td class="value">N/A</td></tr><tr class="line"><td class="heading">Total</td><td class="value">N/A</td></tr></tbody></table></div></div></div><div class="tax-record"><h3 class="h3">Tax Record</h3><div class="tax-values gray-border"><div class="tax-table totals"><table><tbody><tr><td class="header padded">2017</td><td class="value padded font-weight-bold">$8,016</td></tr></tbody></table></div></div></div></div

In [206]:
feats

['# of Bedrooms: 3',
 '3 Bedrooms on Main Level',
 '# of Baths: 3',
 '2 Baths on Main Level',
 '1 Bath on Lower Level',
 'Living Room: View',
 'Living Room on Main Level',
 'Family Room on Lower Level',
 'Dining Room on Main Level',
 'Combination Living/Dining Room',
 'Den/Bonus Room: 0',
 '# of Fireplace(s): 1, In Living Room',
 'Laundry Appliances: Washer/Dryer, In Garage',
 'Refrigerator, Freezer, Island, Remodeled',
 'Kitchen on Main Level',
 'Floors: Hardwood',
 '# of Rooms (Total): 0',
 '1 Bath on Lower Level, Family Room on Lower Level',
 '3 Bedrooms on Main Level, 2 Baths on Main Level, Living Room on Main Level, Dining Room on Main Level, Kitchen on Main Level',
 'Heating/Cooling: Central Heating',
 'Parking Type: On Site',
 '# of Garage Spaces: 1',
 '# of Parking Spaces: 1',
 'Garage',
 'Parking Access: Side-by-Side',
 'Deck(s)',
 'Monthly Rent: 0.00',
 'Unconfirmed Co-op Fee: 0.00',
 'Zoning: RH-1',
 'View(s): Ocean',
 'Lot Measurement: Square Feet',
 'Per Architect',
 'Deck